to-do:
1. for column: ***construction_year***, 34% of data is missing

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

**Import datasets**

In [2]:
file_location = '../data/raw/'

# import training file
df1 = pd.read_csv(f'{file_location}train.csv')
df1['date_recorded'] = pd.to_datetime(df1['date_recorded'])

# import labels (target-variables) for training file
df2 = pd.read_csv(f'{file_location}trainlabels.csv')
df2.columns = ['id', 'target_var']

# merge training file and it's labels (target variable)
df = pd.merge(df1, df2)

data dictionary: https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/page/25/

**Data Wrangling**

In [3]:
df.shape

(59400, 41)

check if the column: id is unique

In [4]:
df['id'].is_unique

True

Assign appropriate datatypes

In [5]:
# convert datatype of cols from int to str
cols = ['id','region_code','district_code',]
for col in cols:
    df[col] = df[col].astype('str')

In [6]:
# look at the first row, along with datatype and # of unique values in each column
pd.concat([df.iloc[0].T,df.dtypes, df.nunique()], 
          axis = 1, 
          keys = ['first row','data type','cardinality'])

,first row,data type,cardinality
id,69572,object,59400
amount_tsh,6000,float64,98
date_recorded,2011-03-14 00:00:00,datetime64[ns],356
funder,Roman,object,1897
gps_height,1390,int64,2428
installer,Roman,object,2145
longitude,34.9381,float64,57516
latitude,-9.85632,float64,57517
wpt_name,none,object,37400
num_private,0,int64,65


### 1. Data Cleaning

drop duplicate rows

In [7]:
print(df.shape)
df.drop_duplicates(inplace = True)
print(df.shape)

(59400, 41)
(59400, 41)


In [8]:
print('Missing: ',df['num_private'].isna().sum())
print(df['num_private'].value_counts(normalize = True))

Missing:  0
0      0.987256
6      0.001364
1      0.001229
5      0.000774
8      0.000774
         ...   
180    0.000017
213    0.000017
23     0.000017
55     0.000017
94     0.000017
Name: num_private, Length: 65, dtype: float64


Drop column: ***num_private*** since 98.7% of the data is **0**s.

In [9]:
df.drop('num_private',axis = 1, inplace = True)

### 2. Missing Values

In [10]:
df.isna().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_group                0
quantity

only show columns where instances of missing value > 0

In [11]:
miss = df.isna().sum()[df.isna().sum() > 0] 

pd.concat([miss, round(miss/df.shape[0]*100,1), df.nunique(), df.dtypes], 
          axis = 1, join = 'inner',
          keys = ['missing','missing %', 'nunique()', 'datatype'])\
          .sort_values(by = 'missing %',
                       ascending = False)

,missing,missing %,nunique(),datatype
scheme_name,28166,47.4,2696,object
scheme_management,3877,6.5,12,object
installer,3655,6.2,2145,object
funder,3635,6.1,1897,object
public_meeting,3334,5.6,2,object
permit,3056,5.1,2,object
subvillage,371,0.6,19287,object


Observation:
1. There are 7 out of 40 (not including target) features with missing data.
2. Among the 7 features, 5 are string, and 2 are boolean

Step:
No data imputation methods applied to deal with missing data in  string/object/categorical variables/features.

---

Among numeric features, column: ***construction_year*** has missing values encoded as **0**s.

In [12]:
df['construction_year'].value_counts(normalize = True).head()

0       0.348636
2010    0.044529
2008    0.043990
2009    0.042643
2000    0.035202
Name: construction_year, dtype: float64

34.86% of the observations are **0**s

range

In [13]:
print(df[df['construction_year']!=0]['construction_year'].min(),
      ' : ',
      df['construction_year'].max())

1960  :  2013


***HAVE TO DEAL WITH MISSING VALUE HERE***

## Export data

In [26]:
df.reset_index(drop = False).to_csv('../data/interim/df.csv', index = False)